<a href="https://colab.research.google.com/github/MrCarry123/OrganSegmentation/blob/main/CNNTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
import os
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Organsegmentation2/') #change this path

Mounted at /content/drive


In [ ]:
 !pip3 install tensorflow==2.9.3
 !pip3 install keras==2.9.0
 !pip3 install -U segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 55.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: ten

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow.keras

%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
tf.keras.backend.set_image_data_format('channels_last')

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [ ]:
import segmentation_models as sm
from glob import glob
import tensorflow as tf
import nibabel as nib
from skimage.transform import resize
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, concatenate, BatchNormalization, Activation, MaxPool2D
import numpy as np
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
# from nilearn.image import resample_img
from tensorflow.keras.utils import to_categorical
import os
import time
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
# Uncomment to select the model
#from resnet50_unet import build_resnet50_unet
#from vgg16_unet import build_vgg16_unet
from vgg19_unet import build_vgg19_unet

In [ ]:
# names = os.listdir('/hdd_storage/data/sumitk/ct_org/train_labels/')

In [ ]:
import cv2

In [ ]:
def np_load_and_preprocess(file_path):
    original_image = cv2.imread(str(file_path.decode()))
    # print('original:', original_image.shape)
    # Resize the image to (32, 32) and convert it to (32, 32, 3)
    resized_image = cv2.resize(original_image, (32, 32)).astype(np.float32)
    original_label = cv2.imread(str(file_path.decode()).replace('images', 'labels').replace('volume', 'labels'))

    # Resize the image to (32, 32) and convert it to (32, 32, 3)
    resized_label = cv2.resize(original_label, (32, 32)).astype(np.float32)
    resized_label[resized_label != 0] = 1
    # print('check: ', resized_image.shape, resized_label.shape)
    return resized_image, resized_label

In [ ]:
# Load and preprocess a single file
def load_and_preprocess_file(file_path):
    # Use tf.numpy_function to wrap the function and use NumPy functions inside TensorFlow
    images, labels = tf.numpy_function(np_load_and_preprocess, [file_path], [tf.float32, tf.float32])
    # images = preprocess_input(images)
    # images = images[:, :,63]
    # labels = labels[:, :,63]
    return images, labels

In [ ]:
# change path
# remove [:100] for training with full dataset
volume_files = glob('/content/drive/MyDrive/Organsegmentation2/train_images/*.png')
# random.shuffle(volume_files)
#volume_files = volume_files[:10]

In [ ]:
len(volume_files)

53760

In [ ]:
train_files, val_files = train_test_split(volume_files, test_size=0.2, random_state=42)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_files).shuffle(buffer_size=len(train_files))
val_dataset = tf.data.Dataset.from_tensor_slices(val_files)

In [ ]:
# Map the load_and_preprocess_file function to the dataset
train_dataset = train_dataset.map(load_and_preprocess_file, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = tf.data.Dataset.from_tensor_slices(val_files)
val_dataset = val_dataset.map(load_and_preprocess_file, num_parallel_calls=tf.data.AUTOTUNE)
# In[14]:

In [ ]:
# Shuffle, batch, cache, and prefetch the dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(train_files))
train_dataset = train_dataset.batch(batch_size)
#train_dataset = train_dataset.cache()
#train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# for step, (x,y) in enumerate(train_dataset):
#     print(x)
#     print(y)

In [ ]:
val_dataset = val_dataset.shuffle(buffer_size=len(val_files))
val_dataset = val_dataset.batch(batch_size)
#val_dataset = val_dataset.cache()
#val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Uncomment to use Custom U-Net model

# def unet(label_num=4, pretrained_weights=None):

#     inputs = Input((32, 32, 4))

#     '''downsample'''
#     conv1 = Conv2D(8, 3, padding='same', kernel_initializer='he_normal')(inputs)
#     batc1 = BatchNormalization(axis=-1)(conv1)
#     acti1 = Activation('relu')(batc1)
#     conv2 = Conv2D(16, 3 , padding='same', kernel_initializer='he_normal')(acti1)
#     batc2 = BatchNormalization(axis=-1)(conv2)
#     acti2 = Activation('relu')(batc2)
#     maxp1 = MaxPooling2D(2)(acti2)


#     conv3 = Conv2D(16, 3, padding='same', kernel_initializer='he_normal')(maxp1)
#     batc3 = BatchNormalization(axis=-1)(conv3)
#     acti3 = Activation('relu')(batc3)
#     conv4 = Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(acti3)
#     batc4 = BatchNormalization(axis=-1)(conv4)
#     acti4 = Activation('relu')(batc4)
#     maxp2 = MaxPooling2D(2)(acti4)


#     conv5 = Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(maxp2)
#     batc5 = BatchNormalization(axis=-1)(conv5)
#     acti5 = Activation('relu')(batc5)
#     conv6 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(acti5)
#     batc6 = BatchNormalization(axis=-1)(conv6)
#     acti6 = Activation('relu')(batc6)
#     maxp3 = MaxPooling2D(2)(acti6)

#     conv7 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(maxp3)
#     batc7 = BatchNormalization(axis=-1)(conv7)
#     acti7 = Activation('relu')(batc7)
#     conv8 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(acti7)
#     batc8 = BatchNormalization(axis=-1)(conv8)
#     acti8 = Activation('relu')(batc8)


#     '''upsample'''
#     upsa1 = UpSampling2D(2)(acti8)
#     # print('upsam1 shape: ', upsam1.shape)
#     # Calculate the cropping amount for conv6
#     # crop_amount = 1  # Adjust this calculation as needed based on your specific requirements

#     # # # Crop conv6 to match the dimensions of upsa1
#     # # cropped_conv6 = Cropping3D(cropping=((0, 0), (0, 0), (crop_amount, crop_amount)))(conv6)
#     # sliced_conv6 = conv6[:, crop_amount:-crop_amount, crop_amount:-crop_amount, crop_amount:-crop_amount, :]

#     # print(tf.shape(conv6[:, :, :, :16, :]))
#     # print(tf.shape(upsa1))
#     merg1 = Concatenate(axis=-1)([conv6, upsa1])

#     # merg1 = Concatenate(axis=-1)([conv6[:, :, :, :16, :], upsa1])
#     conv9 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(merg1)
#     batc9 = BatchNormalization(axis=-1)(conv9)
#     acti9 = Activation('relu')(batc9)
#     conv10 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(acti9)
#     batc10 = BatchNormalization(axis=-1)(conv10)
#     acti10 = Activation('relu')(batc10)

#     upsa2 = UpSampling2D(2)(acti10)
#     merg2 = Concatenate(axis=-1)([conv4, upsa2])
#     conv11 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(merg2)
#     batc11 = BatchNormalization(axis=-1)(conv11)
#     acti11 = Activation('relu')(batc11)
#     conv12 = Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(acti11)
#     batc12 = BatchNormalization(axis=-1)(conv12)
#     acti12 = Activation('relu')(batc12)
#     # print(tf.shape(acti12))
#     upsa3 = UpSampling2D(2)(acti12)
#     # print(tf.shape(upsa3))
#     # print(tf.shape(conv2))
#     merg3 = Concatenate(axis=-1)([conv2, upsa3])
#     conv13 = Conv2D(16, 3, padding='same', kernel_initializer='he_normal')(merg3)
#     batc13 = BatchNormalization(axis=-1)(conv13)
#     acti13 = Activation('relu')(batc13)
#     conv14 = Conv2D(16, 3, padding='same', kernel_initializer='he_normal')(acti13)
#     convol = Conv2D(label_num, 1, padding='same',activation='sigmoid')(conv14)


#     model = Model(inputs=inputs, outputs=convol)
#     #model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef])
#     # model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef])
#     #model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=[dice_coef])
#     #model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#     # model.summary()

#     if (pretrained_weights):
#         model.load_weights(pretrained_weights)

#     return model
# model = unet(label_num=4, pretrained_weights=None)

In [ ]:
# Uncomment to select the model
#model = build_resnet50_unet((32, 32, 3))
#model = build_vgg16_unet((32, 32, 3))
model = build_vgg19_unet((32, 32, 3))

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
model.summary()
num_layers = len(model.layers)
print(num_layers)

Model: "VGG19_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input_2[0][0]']                
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['block1_conv1[0][0]']           
                                                                                                  
 block1_pool (MaxPooling2D)     (None, 16, 16, 64)   0           ['block1_conv2[0][0]']           
                                                                                        

In [ ]:
opt = Adam(learning_rate=0.0001)

In [ ]:
def DiceBCELoss(targets, inputs):
    smooth = 1e-6
    inputs = tf.keras.backend.flatten(inputs)
    targets = tf.keras.backend.flatten(targets)

    BCE = tf.keras.losses.binary_crossentropy(targets, inputs)
    intersection = tf.reduce_sum(targets * inputs)
    dice_loss = 1.0 - (2.0 * intersection + smooth) / (tf.reduce_sum(targets) + tf.reduce_sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    return Dice_BCE

In [ ]:
epochs = 15
from tqdm import tqdm
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Training loop
    for step, (x, y) in tqdm(enumerate(train_dataset)):
        # print(x.shape, y.shape)
        with tf.GradientTape(persistent=True) as tape:
            pred = model(x, training=True)
            loss_value = DiceBCELoss(y, pred)

        grads = tape.gradient(loss_value, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))

    # Validation phase
    val_loss = 0.0
    val_steps = 0
    for step, (val_x, val_y) in tqdm(enumerate(val_dataset)):
        val_pred = model(val_x, training=False)
        val_loss += DiceBCELoss(val_y, val_pred)
        val_steps += 1
    avg_val_loss = val_loss / val_steps

    print("Epoch: %d, Training Loss: %.4f, Validation Loss: %.4f, Time: %.2f seconds" % (epoch, loss_value.numpy(), avg_val_loss, time.time() - start_time))


Start of epoch 0


1344it [41:14,  1.84s/it]
336it [09:29,  1.70s/it]


Epoch: 0, Training Loss: 0.3444, Validation Loss: 0.2565, Time: 3044.71 seconds

Start of epoch 1


1344it [03:07,  7.16it/s]
336it [00:29, 11.40it/s]


Epoch: 1, Training Loss: 0.1227, Validation Loss: 0.1350, Time: 217.09 seconds

Start of epoch 2


...

Epoch: 12, Training Loss: 0.0562, Validation Loss: 0.0481, Time: 214.67 seconds

Start of epoch 13


1344it [03:06,  7.22it/s]
336it [00:29, 11.55it/s]


Epoch: 13, Training Loss: 0.0431, Validation Loss: 0.0478, Time: 215.38 seconds

Start of epoch 14


1344it [03:05,  7.23it/s]
336it [00:29, 11.51it/s]

Epoch: 14, Training Loss: 0.0367, Validation Loss: 0.0447, Time: 215.13 seconds


In [ ]:
model.save('sm6.h5')

In [ ]:
# !jupyter nbconvert --to  script 'sm.ipynb'

In [ ]:
# # load your data


# # preprocess input
# x_train = preprocess_input(x_train)
# x_val = preprocess_input(x_val)

# # define model
# model = sm.Unet('resnet34', classes=7, activation='softmax')
# model.compile(
#     'Adam',
#     loss=sm.losses.bce_jaccard_loss,
#     metrics=[sm.metrics.iou_score],
# )

# # fit model
# # if you use data generator use model.fit_generator(...) instead of model.fit(...)
# # more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
# model.fit(
#    x=x_train,
#    y=y_train,
#    batch_size=16,
#    epochs=100,
#    validation_data=(x_val, y_val),
# )